## [12-27] 流平衡写的是有问题的

In [1210]:
import gurobipy as gb
from gurobipy import*
import openpyxl  
import numpy as np
import math
import time
import copy
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import heapq


In [1211]:
# PARA
M = 10E5
ISSme = 1

In [1212]:
def get_values(filename,id = 0,delme = 0):
    wb = openpyxl.load_workbook(filename)
    sheet_names = wb.sheetnames
    print(sheet_names)

    sheet = wb[sheet_names[id]]
    maxrow = sheet.max_row 
    maxcol = sheet.max_column 
    value = sheet.cell(row=2, column=1).value
    alldata = []

    for row in sheet.iter_rows(min_row=1, max_row=maxrow, min_col=1, max_col=maxcol): # type: ignore
        row_data = [cell.value for cell in row]
        alldata.append(row_data)
    station_name = alldata[0][1:]

    if delme:
        del alldata[0]
        for i in range(len(alldata)):
            del alldata[i][0]


    return alldata,station_name

In [1213]:
def refineme(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] is None:
                data[i][j] = M
    return data

In [1214]:
def convme(data,title):
    degree = {}
    alldegree = 0
    for i,row in enumerate(data):
        
        degree[title[i]] = sum(row)
        alldegree = alldegree + sum(row)
    return degree,alldegree


In [1215]:
def findme_maxdegreee(degree):
    max_degree = max(degree.values())

    max_degree_node = []
    for key in degree:
        if int(degree[key]) == int(max_degree):
            max_degree_node.append(key)
    return(max_degree,max_degree_node)

In [1216]:
def picme(station_name,MyOD):
    plt.figure(figsize=(10, 6))
    MyOD = np.array(MyOD)
    plt.imshow(MyOD, cmap='Blues', aspect='auto', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xticks(ticks=np.arange(len(station_name)), labels=station_name, rotation=45)
    plt.yticks(ticks=np.arange(MyOD.shape[0]), labels=[f'Row {i+1}' for i in range(MyOD.shape[0])])
    plt.title('Heatmap of Matrix')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.tight_layout()
    plt.show()

In [1217]:
MyOD,station_name = get_values('smallOD.xlsx',delme=1)

['Sheet1']


In [1218]:
MyOD = refineme(MyOD)

In [1219]:
station_name.insert(0,'d')
station_name.append('a')

In [1220]:
station_name

['d', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'a']

In [1221]:
# 设置插入的值
value_to_insert = 100

# 示例数据


# 插入新列（在每行的开头和结尾都插入100）
for row in MyOD:
    row.insert(0, value_to_insert)  # 插入开头的列
    row.append(value_to_insert)     # 插入结尾的列

# 打印结果
print("After inserting columns:")
for row in MyOD:
    print(row)

print('\n')

# 插入新行（在开头和结尾插入100的行）
new_row = [value_to_insert for _ in range(len(MyOD[0]))]  # 创建一个独立的新行
MyOD.insert(0, new_row)  # 在开头插入新行
MyOD.append([value_to_insert for _ in range(len(MyOD[0]))])  # 在结尾插入独立的新行

# 打印结果
print("After inserting rows:")
for row in MyOD:
    print(row)


After inserting columns:
[100, 1000000.0, 1000000.0, 1000000.0, 1, 1, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 100]
[100, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]


After inserting rows:
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1, 1, 1000000.0, 1000000.0, 1000000.0, 100]
[100, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1

In [1222]:
MyOD == copy.deepcopy(MyOD)

True

In [1223]:
for i, row in enumerate(MyOD):
    for j, item in enumerate(row):
        # 第一行第一个元素
        if i == 0 and j == 0:
            print(i, j)
            MyOD[i][j] = M

        # 第一行最后一个元素
        elif i == 0 and j == len(row) - 1:
            MyOD[i][j] = M

        # 第一列或最后一列的元素
        elif j == 0 :
            MyOD[i][j] = M

        # 最后一行的元素
        elif i == len(MyOD) - 1:
            print(i, j)
            MyOD[i][j] = M


0 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9


In [1224]:
for row in MyOD:
    print(id(row))

1909546189120
1909546002432
1909565525632
1909539006208
1909565534976
1909564923072
1909565534592
1909546180544
1909546182528
1909546155008


In [1225]:
# 打印结果
for row in MyOD:
    print(row)

[1000000.0, 100, 100, 100, 100, 100, 100, 100, 100, 1000000.0]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 100]
[1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 100]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0]


In [1226]:
model1 = gb.Model()

In [1227]:
x = {}
y = {}

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            x[i,j] = model1.addVar(vtype=gb.GRB.INTEGER,lb = 0,name = f'[x{station_name[i]},{station_name[j]}]')
            y[i,j] = model1.addVar(vtype=gb.GRB.BINARY,lb = 0,name = f'[y{station_name[i]},{station_name[j]}]')

In [1228]:
model1.write("model.lp")

In [1229]:
# flow consit

In [1230]:
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1 ))== quicksum(x[j,len(MyOD)-1]  for j in range(1,len(MyOD)-1,1) ))

<gurobi.Constr *Awaiting Model Update*>

aqws

In [1231]:
for j in range(1,len(MyOD)-1,1):

    model1.addConstr(quicksum(x[i,j] for i in range(len(MyOD)) if i!= j)== quicksum(x[j,k] for k in range(len(MyOD) )if k!= j))

In [1232]:
# node must visit
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [1233]:
model1.addConstr(quicksum(x[k,len(MyOD)-1] for k in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [1234]:
for i in range(1,len(MyOD)-1,1 ):
    print(i)
    model1.addConstr(quicksum(x[k,i] for k in range(len(MyOD)) if k !=i) >= 1)


1
2
3
4
5
6
7
8


In [1235]:
# path one direction
for i in range(0,len(MyOD)-1,1 ):
    for j in range(0,len(MyOD)-1,1 ):
        if i < j:
            model1.addConstr(x[i,j]<= len(MyOD)*len(MyOD)*y[i,j])
            model1.addConstr(x[j,i]<=len(MyOD)*len(MyOD)*(1-y[i,j]))
        

In [1236]:
#main
Z1 = quicksum(int(MyOD[i][j])*x[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)
print(type(Z1))
model1.setObjective(Z1, sense=gb.GRB.MINIMIZE)

<class 'gurobipy.LinExpr'>


In [1237]:
model1.update()

In [1238]:
model1.write("model.lp")

In [1239]:
model1.display()

Minimize
100.0 [xd,V1] + 100.0 [xd,V2] + 100.0 [xd,V3] + 100.0 [xd,V4] + 100.0 [xd,V5]
+ 100.0 [xd,V6] + 100.0 [xd,V7] + 100.0 [xd,V8] + 1000000.0 [xd,a] + 1000000.0 [xV1,d]
+ 1000000.0 [xV1,V2] + 1000000.0 [xV1,V3] + [xV1,V4] + [xV1,V5] + 1000000.0 [xV1,V6]
+ 1000000.0 [xV1,V7] + 1000000.0 [xV1,V8] + 100.0 [xV1,a] + 1000000.0 [xV2,d]
+ 1000000.0 [xV2,V1] + 1000000.0 [xV2,V3] + 1000000.0 [xV2,V4] + 1000000.0 [xV2,V5]
+ 1000000.0 [xV2,V6] + 1000000.0 [xV2,V7] + 1000000.0 [xV2,V8] + 100.0 [xV2,a]
+ 1000000.0 [xV3,d] + 1000000.0 [xV3,V1] + [xV3,V2] + 1000000.0 [xV3,V4]
+ 1000000.0 [xV3,V5] + 1000000.0 [xV3,V6] + 1000000.0 [xV3,V7] + 1000000.0 [xV3,V8]
+ 100.0 [xV3,a] + 1000000.0 [xV4,d] + 1000000.0 [xV4,V1] + 1000000.0 [xV4,V2]
+ 1000000.0 [xV4,V3] + 1000000.0 [xV4,V5] + 1000000.0 [xV4,V6] + 1000000.0 [xV4,V7]
+ 1000000.0 [xV4,V8] + 100.0 [xV4,a] + 1000000.0 [xV5,d] + 1000000.0 [xV5,V1]
+ 1000000.0 [xV5,V2] + 1000000.0 [xV5,V3] + 1000000.0 [xV5,V4] + 1000000.0 [xV5,V6]
+ 1000000.0 [xV5,V7

C:\Users\zhuangz giant\AppData\Local\Temp\ipykernel_10068\2040429350.py:1: DeprecationWarning: Model.display() is deprecated
  model1.display()


In [1240]:
print('\n')
model1.setParam(gb.GRB.Param.MIPGap, 0.005)
model1.setParam(gb.GRB.Param.TimeLimit, 3600*2)
model1.optimize()
# 可以输出求解的决策变量
try:
    if model1.ObjVal >= M*2:
        print(f'Model result of {model1.ObjVal:.2f}  exceed M:{M:.2f}')
        print('Result may out of node\\arc ability ')
    else:
        print(model1.ObjVal)
except Exception as e:
    print(e)
    print('\033[91m\nFatal! Model is INFEASIBLE.')

    if ISSme:
        model1.computeIIS()
        model1.write("model1.ilp")
    else:
        print('\033[5m\n ISS CHECK CANCEL \n\033[0m')



Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-14600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 91 rows, 180 columns and 392 nonzeros
Model fingerprint: 0x9beccfdd
Variable types: 0 continuous, 180 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 8000201.0000
Presolve removed 0 rows and 56 columns
Presolve time: 0.00s
Presolved: 91 rows, 124 columns, 392 nonzeros
Variable types: 0 continuous, 124 integer (36 binary)

Root relaxation: objective 8.040000e+02, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 

In [1241]:
#print(p[reflector[i], reflector[j],k].varName,p[reflector[i], reflector[j],k].x)

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            if x[i,j].x >= 0.001 :
                if MyOD[i][j] > 101:
                    print('OD Shhould not choose ',x[i,j].varName,x[i,j].x)
                else:
                    print(x[i,j].varName,x[i,j].x,x[i,j].x*MyOD[i][j])

[xd,V1] 1.0 100.0
[xd,V3] 1.0 100.0
[xd,V4] 1.0 100.0
[xd,V6] 1.0 100.0
[xV1,V5] 1.0 1.0
[xV2,a] 1.0 100.0
[xV3,V2] 1.0 1.0
[xV4,a] 1.0 100.0
[xV5,V8] 1.0 1.0
[xV6,V7] 1.0 1.0
[xV7,a] 1.0 100.0
[xV8,a] 1.0 100.0
